# 투빅스 18기 정규세션 Week1 EDA - 이정우

# 1. 라이브러리 Import & 하이퍼파라미터 설정



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

## GPU 설정 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.is_available(): cuda가 사용 가능하면 true를 반환
# gpu가 사용 가능한 경우에는 device를 gpu로 설정하고 불가능하면 cpu로 설정

# Pytorch random seed 고정
torch.manual_seed(777) 
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

print(device + " is available") # 'cuda' or 'cpu' 
 
## 하이퍼파라미터 값 설정 
learning_rate = 0.001  # 학습률
batch_size = 100       # 배치 크기 
num_classes = 10       # 클래스 수
epochs = 5             # 에포크

cuda is available


# 2. 데이터셋 불러오기

In [2]:
## TorchVision에서 MNIST 데이터셋 불러오기
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',               # 학습/테스트 데이터가 저장되는 경로
    train = True,                        # 학습용/테스트용 데이터셋 여부 지정
    download = True,                     # root에 데이터가 없는 경우 인터넷에서 다운로드
    transform = transfroms.Compose([     # 특징(feature)의 변형(transform) 방식 지정
        transfroms.ToTensor()            
    ])
)
# transforms.ToTensor(): 데이터를 tensor로 변형
# transforms.Compose() : 여러 단계로 변환해야 하는 경우, Compose를 통해 여러 단계를 묶을 수 있음

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



# 3. DataLoader로 학습용 데이터 준비하기

In [3]:
# DataLoader란? 
'''
Dataset을 batch기반의 딥러닝모델 학습을 위해서 미니배치 형태로 만들어서 우리가 실제로 학습할 때 이용할 수 있게 형태를 만들어주는 기능이다.
Dataset의 전체 데이터가 batch size로 slice되어 공급된다.
DataLoader는 iterator 형식으로 데이터에 접근 하도록 하며 batch_size나 shuffle 유무를 설정할 수 있다.

출처 : https://wikidocs.net/156998 [PyTorch 딥러닝 챗봇]
'''

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
# shuffle 옵션 추가: Epoch 마다 데이터셋을 섞어, 데이터가 학습되는 순서를 바꿀 때 사용 (train => True, test => False)
 
examples = enumerate(train_set) # 인덱스 부여
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape # 데이터 사이즈 확인

torch.Size([1, 28, 28])

# 4. 모델 정의

In [4]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()
        # 신경망 layer 구성 요소 정의

        # in_channels = 1, out_channels = 10, kernel size = 5, stribe = 1, padding = 0인 Conv2d
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        # in_channels = 10, out_channels = 20, kernel size = 5, stribe = 1, padding = 0인 Conv2d
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        # layer node의 25%는 dropout 적용 (연결 끊기)
        # p = 0.25, iplace = False인 Dropout
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        # kernel_size = 2인 Maxpool
        self.mp = nn.MaxPool2d(2) 
        # in_features = 320, out_features = 100인 Linear 
        self.fc1 = nn.Linear(320,100) 
        # in_features = 100, out_features = 10인 Linear 
        self.fc2 = nn.Linear(100,10) 

  # 순전파 함수 정의: Network 동작
  def forward(self, x):
        # Convolutional layer
        # conv - pooling - relu (하나의 묶음)
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        # dropout 적용
        x = self.drop2D(x)
        # 데이터를 fc layer에 넣기 위해 1차원 형태로 변경 (flatten)
        x = x.view(x.size(0), -1) 
        # Fully connected layer
        x = self.fc1(x) 
        x = self.fc2(x) 
        return F.log_softmax(x) 


# 5. 모델 & Loss function & Optimizer 설정


In [5]:
# 모델 클래스 인스턴스화, 지정한 장치에 올림
model = ConvNet().to(device)

# 손실함수(loss function) 및 최적화함수(optimizer)를 설정 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# 6. 모델 학습


In [6]:
## Train
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        # 모든 model의 gradient 값을 0으로 설정 
        # 초기화 이유 : Pytorch에서는 미분을 통해 얻은 gradients값들을 추후에 backward를 해줄 때 계속 누적시키기 때문에
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()  # 역전파(gradients 저장)
        optimizer.step() # 파라미터 update 진행 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch:    1] cost = 0.316490054
[Epoch:    2] cost = 0.117380545
[Epoch:    3] cost = 0.088905476
[Epoch:    4] cost = 0.0759488344
[Epoch:    5] cost = 0.0652774945


# 7. 모델 성능 평가



In [7]:
## Evaluation/Validation
# Train mode => Eval mode로 변경 
model.eval()
# 학습할 때만 필요했던 기능을 비활성화
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy:  98.48 %
